In [31]:
import sys
sys.path.append('../')

import gc
import numpy as np
from scipy import ndimage

from skimage.filters import sobel_h
from skimage.filters import sobel_v
from scipy import stats

#from sa_decomp_layer import SADecompLayer



import os
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec


#tf.__version__
from tensorflow.python.client import device_lib
#print(device_lib.list_local_devices())

plt.rcParams['figure.figsize'] = [10,10]

import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import decode_predictions
from tensorflow.keras.applications import VGG16, ResNet152, ResNet50

from tensorflow.nn import depthwise_conv2d
from tensorflow.math import multiply, reduce_sum, reduce_euclidean_norm, sin, cos, abs
from tensorflow import stack, concat, expand_dims

import tensorflow_probability as tfp
from glob import glob


from tensorflow.keras.utils import to_categorical
from tensorflow.data import Dataset, AUTOTUNE
from imagenet_utils.imagenet_clsloc2 import clsloc
from imagenet_utils.load_images import load_images
from imagenet_utils.preprocess import preprocess

from utils.utils import *

import keras
from keras import layers

from keras import backend as K

import scienceplots

plt.style.use(['science', 'ieee'])
plt.rcParams.update({'figure.dpi': '600'})

In [32]:
gc.collect()
model = VGG16(weights='imagenet',
				  include_top=True,
				  input_shape=(224, 224, 3))

model.compile(loss=keras.losses.CategoricalCrossentropy(from_logits=False),            
			metrics=[
				keras.metrics.CategoricalAccuracy(name="accuracy"),
				keras.metrics.TopKCategoricalAccuracy(5, name="top-5-accuracy"),])


conv_layers = []
for l in model.layers:
	if 'conv2d' in str(type(l)).lower():
		if l.kernel_size == (3,3) or l.kernel_size == (7,7):
			conv_layers.append(l)
			print(l)

In [42]:
def rotate (model_original, layers)  :
	model = tf.keras.models.clone_model(model_original)
	model.set_weights(model_original.get_weights())
	conv_layers = []
	for l in model.layers:
		if 'conv2d' in str(type(l)).lower():
			if l.kernel_size == (3,3) or l.kernel_size == (7,7):
				conv_layers.append(l)
				print(l)
	for l in layers:
		filters, biases = conv_layers[l].get_weights()


		# Generate random rotations (0, 1, 2, 3 corresponding to 0°, 90°, 180°, 270°)
		random_rotations = tf.random.uniform((1,1,filters.shape[-2]*filters.shape[-1]), minval=2, maxval=3, dtype=tf.int32) * tf.ones((3,3,filters.shape[-2]*filters.shape[-1]), dtype=tf.int32)

		# Reshape weights to (64 * 128, 3, 3) for batch processing
		weights_reshaped = tf.reshape(filters, (3, 3,-1))
		random_rotations *= tf.where(tf.less(tf.reduce_mean(weights_reshaped, axis=(0,1)), 0), x=0, y=1)
		rotation_masks = [tf.equal(random_rotations, k) for k in [0,2]] 
		print(random_rotations)
		# Apply rotations based on the masks
		rotated_batches = [
			tf.image.rot90(weights_reshaped, k=k) * tf.cast(rotation_masks[k//2], tf.float32)
			for k in [0,2]
		]

		rotated_weights = tf.add_n(rotated_batches)
		rotated_weights = tf.reshape(rotated_weights, (3, 3,filters.shape[-2],filters.shape[-1]))
		
		conv_layers[l].set_weights([rotated_weights, biases])


	return model

		




In [43]:
# Imagenet Validation Set location
img = "C:/ILSVRC2012_img_val/ILSVRC2012_val_00000028.JPEG"

In [44]:
train_list = glob("C:/imagenette2/train/*/*.JPEG")
train_labels = glob("C:/imagenette2/train/*/*.JPEG")
train_labels = to_categorical([clsloc[os.path.normpath(str(path)).split(os.path.sep)[-2]] for path in train_labels], num_classes = 1000)


trainDS = Dataset.from_tensor_slices((train_list, train_labels))
trainDS = (trainDS
	.map(load_images)
	.map(preprocess)
	.cache()
	.batch(1)
	.prefetch(AUTOTUNE)
)


val_list = glob("C:/imagenette2/val/*/*.JPEG")
val_labels = glob("C:/imagenette2/val/*/*.JPEG")
val_labels = to_categorical([clsloc[os.path.normpath(str(path)).split(os.path.sep)[-2]] for path in val_list], num_classes = 1000)


valDS = Dataset.from_tensor_slices((val_list, val_labels))
valDS = (valDS
	.map(load_images)
	.map(preprocess)
	.cache()
	.batch(32)
	.prefetch(AUTOTUNE)
)

In [45]:
val_labels.shape

(3925, 1000)

In [46]:
model_rot = rotate(model, [-3,-2,-1])
model_rot.compile(loss=keras.losses.CategoricalCrossentropy(from_logits=False),            
			metrics=[
				keras.metrics.CategoricalAccuracy(name="accuracy"),
				keras.metrics.TopKCategoricalAccuracy(5, name="top-5-accuracy"),])

tf.Tensor(
[[[0 0 2 ... 0 0 2]
  [0 0 2 ... 0 0 2]
  [0 0 2 ... 0 0 2]]

 [[0 0 2 ... 0 0 2]
  [0 0 2 ... 0 0 2]
  [0 0 2 ... 0 0 2]]

 [[0 0 2 ... 0 0 2]
  [0 0 2 ... 0 0 2]
  [0 0 2 ... 0 0 2]]], shape=(3, 3, 262144), dtype=int32)
tf.Tensor(
[[[0 0 0 ... 0 2 2]
  [0 0 0 ... 0 2 2]
  [0 0 0 ... 0 2 2]]

 [[0 0 0 ... 0 2 2]
  [0 0 0 ... 0 2 2]
  [0 0 0 ... 0 2 2]]

 [[0 0 0 ... 0 2 2]
  [0 0 0 ... 0 2 2]
  [0 0 0 ... 0 2 2]]], shape=(3, 3, 262144), dtype=int32)
tf.Tensor(
[[[0 2 0 ... 0 0 0]
  [0 2 0 ... 0 0 0]
  [0 2 0 ... 0 0 0]]

 [[0 2 0 ... 0 0 0]
  [0 2 0 ... 0 0 0]
  [0 2 0 ... 0 0 0]]

 [[0 2 0 ... 0 0 0]
  [0 2 0 ... 0 0 0]
  [0 2 0 ... 0 0 0]]], shape=(3, 3, 262144), dtype=int32)


In [47]:
model_rot.evaluate(valDS)

123/123 [==============================] - 23s 184ms/step - loss: 3.1085 - accuracy: 0.3569 - top-5-accuracy: 0.6318


[3.10848069190979, 0.35694268345832825, 0.6318471431732178]

In [48]:
model.evaluate(valDS)

123/123 [==============================] - 12s 98ms/step - loss: 0.6863 - accuracy: 0.8000 - top-5-accuracy: 0.9605


[0.6863359212875366, 0.800000011920929, 0.9605095386505127]